In [43]:
# https://machinelearningmastery.com/time-series-forecasting-with-prophet-in-python/

In [44]:
# check prophet version
import prophet
# print version number
print('Prophet %s' % prophet.__version__)

from pandas import read_csv
from matplotlib import pyplot

from pandas import to_datetime

from prophet import Prophet

from pandas import DataFrame

import pandas as pd
import numpy as np

Prophet 1.0


In [45]:
train_perc = 0.66

In [46]:
# Load metrics

#df_dailysteps=pd.read_csv("Resources\steps_p01.csv",index_col='Dates',parse_dates=True)

df_steps=pd.read_csv("final_merged_all_participants/steps.csv")
df_active_duration=pd.read_csv("final_merged_all_participants/active_duration.csv")
df_active_heart_rate=pd.read_csv("final_merged_all_participants/active_heart_rate.csv")
df_calories=pd.read_csv("final_merged_all_participants/calories.csv")
df_distance=pd.read_csv("final_merged_all_participants/distance.csv")
df_resting_heartrate=pd.read_csv("final_merged_all_participants/resting_heartrate.csv")
df_sleep=pd.read_csv("final_merged_all_participants/sleep.csv")
df_participant_overview=pd.read_csv("final_merged_all_participants/participant_overview.csv")

#"C:\Users\travi\Documents\Challenge 20\final_merged_all_participants\steps.csv"
#"C:\Users\travi\Documents\Challenge 20\final_merged_all_participants\active_duration.csv"
#"C:\Users\travi\Documents\Challenge 20\final_merged_all_participants\active_heart_rate.csv"
#"C:\Users\travi\Documents\Challenge 20\final_merged_all_participants\calories.csv"
#"C:\Users\travi\Documents\Challenge 20\final_merged_all_participants\distance.csv"
#"C:\Users\travi\Documents\Challenge 20\final_merged_all_participants\resting_heartrate.csv"
#"C:\Users\travi\Documents\Challenge 20\final_merged_all_participants\sleep.csv"
#"C:\Users\travi\Documents\Challenge 20\final_merged_all_participants\participant_overview.csv"


In [47]:
df_sleep.rename(columns = {"Date_Of_Sleep": "Dates"}, inplace = True)

# Drop the label 'time'
df_active_heart_rate = df_active_heart_rate.drop(columns="Time")

df_participant_IDs = df_participant_overview["Participant_ID"]

In [48]:
def parser(df):
    df.dropna()
    df = df.drop(["Participant_ID"],axis=1)
    df.columns = ['ds', 'y']
    df['ds']= to_datetime(df['ds'])
    
    return df

In [51]:
def evaluate_model(predictions, test):
    mape = np.mean(np.abs(predictions - test)/np.abs(test))  # MAPE.ipynb
    return mape

In [64]:
active_duration_forcast_df = pd.DataFrame(columns = ['Participant_ID', 'Dates', 'Active_Duration_in_mins'])
steps_forcast_df = pd.DataFrame(columns = ['Participant_ID', 'Dates', 'Minutes_Asleep'])
active_heart_rate_forcast_df = pd.DataFrame(columns = ['Participant_ID', 'Dates', 'Avg_ActiveHeartRate'])
distance_forcast_df = pd.DataFrame(columns = ['Participant_ID', 'Dates', 'Distance'])
calories_forcast_df = pd.DataFrame(columns = ['Participant_ID', 'Dates', 'Calories'])
resting_heartrate_forcast_df = pd.DataFrame(columns = ['Participant_ID', 'Dates', 'Resting_beats_per_min'])
sleep_forcast_df = pd.DataFrame(columns = ['Participant_ID', 'Dates', 'Minutes_Asleep'])

In [63]:
for id in df_participant_IDs:
#    print(id)
# loop through Participant IDs
    df_steps_X = df_steps.loc[df_steps["Participant_ID"] == id, :]
    df_active_duration_X = df_active_duration.loc[df_active_duration["Participant_ID"] == id, :]
    df_active_heart_rate_X = df_active_heart_rate.loc[df_active_heart_rate["Participant_ID"] == id, :]
    df_distance_X = df_distance.loc[df_distance["Participant_ID"] == id, :]
    df_calories_X = df_calories.loc[df_calories["Participant_ID"] == id, :]
    df_resting_heartrate_X = df_resting_heartrate.loc[df_resting_heartrate["Participant_ID"] == id, :]
    df_sleep_X = df_sleep.loc[df_sleep["Participant_ID"] == id, :]

# parse data for model    
    df_steps_X = parser(df_steps_X)
    df_active_duration_X = parser(df_active_duration_X)
    df_active_heart_rate_X = parser(df_active_heart_rate_X)
    df_distance_X = parser(df_distance_X)
    df_calories_X = parser(df_calories_X)
    df_resting_heartrate_X = parser(df_resting_heartrate_X)
    df_sleep_X = parser(df_sleep_X)

# set train and test data
    train_size = int(len(df_steps_X) * abs(train_perc))
    df_steps_test, df_steps_train = df_steps_X[0:train_size], df_steps_X[train_size:]

    train_size = int(len(df_active_duration_X) * abs(train_perc))
    df_active_duration_test, df_active_duration_train = df_active_duration_X[0:train_size], df_active_duration_X[train_size:]

    train_size = int(len(df_active_heart_rate_X) * abs(train_perc))
    df_active_heart_rate_test, df_active_heart_rate_train = df_active_heart_rate_X[0:train_size], df_active_heart_rate_X[train_size:]
    
    train_size = int(len(df_distance_X) * abs(train_perc))
    df_distance_test, df_distance_train = df_distance_X[0:train_size], df_distance_X[train_size:]
    
    train_size = int(len(df_calories_X) * abs(train_perc))
    df_calories_test, df_calories_train = df_calories_X[0:train_size], df_calories_X[train_size:]

    train_size = int(len(df_resting_heartrate_X) * abs(train_perc))
    df_resting_heartrate_test, df_resting_heartrate_train = df_resting_heartrate_X[0:train_size], df_resting_heartrate_X[train_size:]
    
    train_size = int(len(df_sleep_X) * abs(train_perc))
    df_sleep_test, df_sleep_train = df_sleep_X[0:train_size], df_sleep_X[train_size:]
    
    
# define the model
    model_steps = Prophet()
    model_active_duration = Prophet()
    model_active_heart_rate = Prophet()
    model_distance = Prophet()
    model_calories = Prophet()
    model_resting_heartrate = Prophet()
    model_sleep = Prophet()
    
# fit the model
    model_steps.fit(df_steps_test)
    model_active_duration.fit(df_active_duration_test)
    model_active_heart_rate.fit(df_active_heart_rate_test)
    model_distance.fit(df_distance_test)
    model_calories.fit(df_calories_test)
    model_resting_heartrate.fit(df_resting_heartrate_test)
    model_sleep.fit(df_sleep_test)
    
# test model

    future = model_steps.make_future_dataframe(periods=365)   
    steps_train_forcast = model_steps.predict(future)
    
    future = model_active_duration.make_future_dataframe(periods=365) 
    active_duration_train_forcast = model_active_duration.predict(future)
    
    future = model_active_heart_rate.make_future_dataframe(periods=365)
    active_heart_rate_train_forcast = model_active_heart_rate.predict(future)
    
    future = model_distance.make_future_dataframe(periods=365)
    distance_train_forcast = model_distance.predict(future)
    
    future = model_calories.make_future_dataframe(periods=365)
    calories_train_forcast = model_calories.predict(future)
    
    future = model_resting_heartrate.make_future_dataframe(periods=365)
    resting_heartrate_train_forcast = model_resting_heartrate.predict(future)
    
    future = model_sleep.make_future_dataframe(periods=365)
    sleep_train_forcast = model_sleep.predict(future)
    

    

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 23.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Di

In [ ]:
# write to CSV files

#    steps_train_forcast.to_csv('folder/subfolder/out.csv')
#    active_duration_train_forcast.to_csv('folder/subfolder/out.csv')
#    active_heart_rate_train_forcast.to_csv('folder/subfolder/out.csv')
#    distance_train_forcast.to_csv('folder/subfolder/out.csv')
#    calories_train_forcast.to_csv('folder/subfolder/out.csv')
#    resting_heartrate_train_forcast.to_csv('folder/subfolder/out.csv')
#    sleep_train_forcast.to_csv('folder/subfolder/out.csv')


In [67]:
steps_train_forcast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2019-11-19,613.400045,775.343846,1781.114083,613.400045,613.400045,674.787980,674.787980,674.787980,674.787980,674.787980,674.787980,0.0,0.0,0.0,1288.188024
1,2019-12-02,832.051967,598.096413,1667.560209,832.051967,832.051967,290.351747,290.351747,290.351747,290.351747,290.351747,290.351747,0.0,0.0,0.0,1122.403714
2,2019-12-06,899.329482,278.817153,1352.815392,899.329482,899.329482,-65.768916,-65.768916,-65.768916,-65.768916,-65.768916,-65.768916,0.0,0.0,0.0,833.560567
3,2019-12-14,1033.884514,222.555045,1292.341272,1033.884514,1033.884514,-253.149208,-253.149208,-253.149208,-253.149208,-253.149208,-253.149208,0.0,0.0,0.0,780.735306
4,2019-12-18,1101.162029,2378.308607,3412.720252,1101.162029,1101.162029,1808.595640,1808.595640,1808.595640,1808.595640,1808.595640,1808.595640,0.0,0.0,0.0,2909.757668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,2021-01-12,7677.539066,7830.610494,8863.248429,7677.537600,7677.540326,674.787980,674.787980,674.787980,674.787980,674.787980,674.787980,0.0,0.0,0.0,8352.327046
372,2021-01-13,7694.358445,8987.681826,9990.101773,7694.356974,7694.359709,1808.595640,1808.595640,1808.595640,1808.595640,1808.595640,1808.595640,0.0,0.0,0.0,9502.954085
373,2021-01-14,7711.177824,6006.293637,6966.261239,7711.176346,7711.179096,-1227.408627,-1227.408627,-1227.408627,-1227.408627,-1227.408627,-1227.408627,0.0,0.0,0.0,6483.769197
374,2021-01-15,7727.997202,7113.923061,8195.044617,7727.995715,7727.998482,-65.768916,-65.768916,-65.768916,-65.768916,-65.768916,-65.768916,0.0,0.0,0.0,7662.228286
